# Efficient Numerical Computation

One of the critical skills for being a data scientist is understanding computation and algorithms.  Below is a (cursory) guide meant to whet your appetite for the computational techniques that I found useful as a data scientist.  Remember, there’s a lifetime of things to learn and these are just some highlights:

**Vectorized Linear Algebra:**  Let’s say you want to dot product (http://en.wikipedia.org/wiki/Dot_product) two very large vectors.  You could use a for loop, but that’s slow.  Instead, consider using vectorized linear algebra that calls out to professional numerical libraries like BLAS (http://www.netlib.org/blas/) or LAPACK (http://www.netlib.org/lapack/):

In [1]:
import numpy as np
x = np.random.randn(1000000)
y = np.random.randn(1000000)

z1 = sum(x[i] * y[i] for i in xrange(len(x)))  # elapsed time: 1.2176 seconds
z2 = np.dot(x, y)  # elapsed time: 0.0045 seconds
print("z1: ", z1)
print("z2: ", z2)

('z1: ', 349.11451587016472)
('z2: ', 349.11451587016381)


Run both samples and see which one takes longer (try using python’s **timeit** module (https://docs.python.org/2/library/timeit.html) if you are not already familiar with it). In our example, for loops are 600 times slower.  You can learn more about numerical computation from the numpy and scipy websites.

In [2]:
import timeit
from timeit import time
%time(sum(x[i] * y[i] for i in xrange(len(x))))
%time(np.dot(x, y))  # elapsed time is a little smaller for this

CPU times: user 533 ms, sys: 99.1 ms, total: 632 ms
Wall time: 557 ms
CPU times: user 967 µs, sys: 23 µs, total: 990 µs
Wall time: 886 µs


349.11451587016381

**Simulation.** The simplest workhorse when trying to get a handle on a complex system is simulation.  For example, consider the problem:You flip a fair coin n times.  What is the expected number of heads?  What is the standard deviation of the number of heads?

Obviously, this is just a **Bernoulli Distribution** (http://en.wikipedia.org/wiki/Bernoulli_distribution) and does not require simulation but we will use it to didactically illustrate what one might do in more complex situations.  The simulation code might be:

In [3]:
np.random.seed(42)
samples = np.random.randint(0,2,10000)
print samples.mean() # 0.4987
print samples.std() # 0.499998309997

0.4987
0.499998309997


Notice how much faster vectorized linear algebra is compared with running python for loops.  Obviously, the field can get very complex, with Dynamical Systems (http://en.wikipedia.org/wiki/Dynamical_system), Monte Carlo (http://en.wikipedia.org/wiki/Monte_Carlo_method), Gibbs Sampling (http://en.wikipedia.org/wiki/Gibbs_sampling), Importance Sampling (http://en.wikipedia.org/wiki/Importance_sampling), and MCMC (http://en.wikipedia.org/wiki/Markov_chain_Monte_Carlo). Don’t forget to use bootstrapping (http://en.wikipedia.org/wiki/Bootstrapping_%28statistics%29) to estimate error bars.

**Recursion.**  Of course, simulations can never give you an exact answer.  One technique to get an exact answer that works in many cases is recursion.  The simplest example of recursion (http://en.wikipedia.org/wiki/Recursion_%28computer_science%29) comes from implementing the Fibonacci sequence (http://en.wikipedia.org/wiki/Fibonacci_number):

In [4]:
def fib(n):
  if n < 2:
    return 1
  else:
    return fib(n-1) + fib(n-2)
%time(fib(5))

CPU times: user 9 µs, sys: 3 µs, total: 12 µs
Wall time: 14.8 µs


8

Try timing the runs to guess the running time (http://en.wikipedia.org/wiki/Time_complexity) of the Fibonacci sequence (spoiler alert: it’s exponential).  You may be surprised by how slow it is (can you guess why?).

In [5]:
%time(fib(19))

CPU times: user 6.21 ms, sys: 3.96 ms, total: 10.2 ms
Wall time: 7.05 ms


6765

To see how we might use this to solve the last problem, notice that on the n-th draw, we can either increase or decrease the average number of heads by 1/n from the n-th draw, and that each occurs with probability 1/2. Here is the recursive code:

In [7]:
def average_heads(n):
  if n == 1:
    return 0.5
  else:
    return np.mean([average_heads(n-1) + 1./n, average_heads(n-1) - 1./n])
%time(average_heads(12))

CPU times: user 48.2 ms, sys: 10.2 ms, total: 58.4 ms
Wall time: 51.1 ms


0.5

Think a little about how you might compute the standard deviation using this technique (hint: it may help to review alternative formulas for variance, https://en.wikipedia.org/wiki/Variance#Definition).  Another popular use of recursion is in graph traversal algorithms (https://en.wikipedia.org/wiki/Graph_traversal).  Consider the question:

How many possible ways are there to draw US coins that sum up to 50 cents?

For the sake of definiteness, we will say that order of the drawn coins matters.  You can solve this problem by traversing the “graph” of all possible draws until we reach exactly 50 cents:

In [9]:
coins = [1, 5, 10, 25, 50]
def count(remainder):
  if remainder < 0:
    return 0
  if remainder == 0:
    return 1
  return sum(count(remainder - coin) for coin in coins)
count(50)
%time(count(50))

CPU times: user 16.6 s, sys: 911 ms, total: 17.5 s
Wall time: 16.8 s


1931846

This is just the tip of the iceberg of what you can do with recursion.  If you are interested, try looking up algorithms like Depth-First Search (https://en.wikipedia.org/wiki/Depth-first_search), Breadth-First Search (http://en.wikipedia.org/wiki/Breadth-first_search), or tail recursion (http://en.wikipedia.org/wiki/Tail_call#Tail_recursion_modulo_cons).

**Memoization and Dynamic Programming.**  Notice that in both the above examples, we make multiple calls to the recursive function for the same input, which seems inefficient.  A common way to speed this up is to remember (or memoize) the results of previous computations.  As a simple example, take a look at the python memoized class (https://wiki.python.org/moin/PythonDecoratorLibrary#Memoize) which uses python decorator syntax (https://www.python.org/dev/peps/pep-0318/). Here it is in action:

In [16]:
import collections
import functools

class memoized(object):
    '''Decorator. Caches a function's return value each time it is called.
    If called later with the same arguments, the cached value is returned
    (not reevaluated).
    '''
    def __init__(self, func):
         self.func = func
         self.cache = {}
    def __call__(self, *args):
         if not isinstance(args, collections.Hashable):
            # uncacheable. a list, for instance.
            # better to not cache than blow up.
            return self.func(*args)
         if args in self.cache:
            return self.cache[args]
         else:
            value = self.func(*args)
            self.cache[args] = value
            return value
    def __repr__(self):
         '''Return the function's docstring.'''
         return self.func.__doc__
    def __get__(self, obj, objtype):
         '''Support instance methods.'''
         return functools.partial(self.__call__, obj)
@memoized
def fib(n):
  if n < 2:
    return 1
  else:
    return fib(n-1) + fib(n-2)
%time(fib(19))

CPU times: user 152 µs, sys: 95 µs, total: 247 µs
Wall time: 218 µs


6765

This has now effectively turned a recursive program into one using Dynamic Programming (http://en.wikipedia.org/wiki/Dynamic_programming).  Try using timing to guess the running time of the Fibonacci sequence (spoiler alert: it’s linear).  It’s amazing how much of a difference a single line makes!

**Divide and Conquer.**  Another common approach is to break up a large problem into smaller subproblems.  A classic example of this is Merge Sort (http://en.wikipedia.org/wiki/Merge_sort).  For the Fibonacci sequence, consider using the Matrix Form (https://en.wikipedia.org/wiki/Fibonacci_number#Matrix_form) of the Fibonacci sequence:


In [18]:
M = np.matrix([[1, 1], [1, 0]])

def fib(n):
  if n < 2:
    return 1
  MProd = M.copy()
  for _ in xrange(n-2):
    MProd *= M
  return MProd[0,0] + MProd[0,1]
%time(fib(19))

CPU times: user 1.67 ms, sys: 2.28 ms, total: 3.95 ms
Wall time: 274 ms


6765

Since the code relies on repeated matrix multiplication, it is very amenable to Divide and Conquer (http://en.wikipedia.org/wiki/Divide_and_conquer_algorithms) techniques (hint: M^8=(((M^2)^2)^2)). We’ll let you write down the algorithm and time it to verify that it is a log algorithm.  (Isn’t it amazing that this can be done in sub-linear time!)

**Coda:** This is it for this tutorial but if you know a little bit about matrix factorization (http://en.wikipedia.org/wiki/Matrix_decomposition#Eigendecomposition), try working out a (pseudo-)constant time answer.  Why isn’t this really constant time?  (Spoiler alert: read a little bit about Arbitrary Precision Arithmetic (http://en.wikipedia.org/wiki/Arbitrary-precision_arithmetic#Implementation_issues).)  We’re not going to emphasize it because the technique isn’t really all that generalizable but it is still fun to think about.
Of course, this is just a very high-level overview that is meant to pique your interest rather than give you a full exposition. 